<a href="https://colab.research.google.com/github/RicAntonio/Pepsico_Data/blob/main/Pepsico_Daily_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PEPSICO DAILY DATA

Predecir el dia que las acciones suben o bajan

In [ ]:
# Instalar cualquier librería adicional si es necesario (normalmente las librerías comunes ya están instaladas)
# !pip install some_library

# Montar Google Drive si necesitas guardar o cargar archivos desde allí
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

# Si el dataset está en Google Drive, puedes accederlo con algo como:
data_filepath = '/content/drive/MyDrive/pepsico_daily_data.csv'

# Definir rutas de archivo
base_path = '/content/drive/My Drive/Pepsico_Project/'
features_file = base_path + 'pepsico_features.csv'
target_file = base_path + 'pepsico_target.csv'
model_file = base_path + 'pepsico_model.pkl'
prediction_file = base_path + 'pepsico_prediction.csv'


Mounted at /content/drive


In [ ]:
import pandas as pd

# Cargar datos diarios de Pepsico desde Google Drive
data = pd.read_csv(base_path + 'pepsico_daily_data.csv')

# Generar características adicionales
data['Moving_Avg_5'] = data['Close'].rolling(window=5).mean()
data['Moving_Avg_10'] = data['Close'].rolling(window=10).mean()
data['Volatility'] = data['Close'].rolling(window=10).std()

# Crear la etiqueta (1 si el precio sube, 0 si baja)
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Eliminar filas con valores NaN generados por las medias móviles
data = data.dropna()

# Guardar características y etiquetas en Google Drive
features = data[['Moving_Avg_5', 'Moving_Avg_10', 'Volatility']]
target = data['Target']
features.to_csv(features_file, index=False)
target.to_csv(target_file, index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle


# Cargar características y etiquetas desde Google Drive
X = pd.read_csv(features_file)
y = pd.read_csv(target_file)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenar un modelo de clasificación
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Guardar el modelo entrenado en Google Drive
with open(model_file, 'wb') as model_pkl:
    pickle.dump(model, model_pkl)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
import pandas as pd
import pickle

# Cargar el modelo entrenado desde Google Drive
with open(model_file, 'rb') as model_pkl:
    model = pickle.load(model_pkl)

# Cargar las características más recientes desde Google Drive
X_new = pd.read_csv(features_file).iloc[-1:]  # Solo las últimas características

# Generar predicciones
prediction = model.predict(X_new)

# Guardar la predicción en Google Drive
pd.DataFrame({'Prediction': prediction}).to_csv(prediction_file, index=False)
